In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import os
import matplotlib.pyplot as plt
import nltk
from collections import OrderedDict

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from collections import OrderedDict

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to /home/riya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/riya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/riya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from keybert import KeyBERT

In [3]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api
import matplotlib.pyplot as plt
import spacy
import string
from sklearn.metrics.pairwise import cosine_similarity

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
#list(gensim.downloader.info()['models'].keys())
#wv = api.load('glove-wiki-gigaword-300')
#wv.save('glove_wiki_vectors.kv')

In [5]:
revalive_path_vec = "../../glove_wiki_vectors.kv"
relative_path_sdg = "../../sdg_data.csv"
relative_path_attr = "../Dictionary/Attributes_2019-20.csv"
relative_string_map = "./Data_Indicator/String_map.csv"
relative_string_map_unroll = "./Data_Indicator/String_map_unroll.csv"
relative_semantic = "./Data_Indicator/Semantic_map.csv"
relative_semantic_unroll = "./Data_Indicator/Semantic_map_unroll.csv"
relative_sem_threshold = "./Data_Indicator/Semantic_threshold_map.csv"
relative_sem_threshold_unroll = "./Data_Indicator/Semantic_threshold_map_unroll.csv"

# Get the absolute path by joining the current directory with the relative path
absolute_path_vec = os.path.normpath(os.path.join(os.getcwd(), revalive_path_vec))
absolute_path_sdg = os.path.normpath(os.path.join(os.getcwd(), relative_path_sdg))
absolute_path_attr = os.path.normpath(os.path.join(os.getcwd(), relative_path_attr))
absolute_path_string = os.path.normpath(os.path.join(os.getcwd(), relative_string_map))
absolute_path_string_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_string_map_unroll))
absolute_path_semantic = os.path.normpath(os.path.join(os.getcwd(), relative_semantic))
absolute_path_semantic_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_semantic_unroll))
absolute_path_sem_threshold =os.path.normpath( os.path.join(os.getcwd(), relative_sem_threshold))
absolute_path_sem_threshold_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_sem_threshold_unroll))

In [6]:
wv = KeyedVectors.load(absolute_path_vec)

In [7]:
kw_model = KeyBERT(model=wv)

In [8]:
def replace_words(df, column_list, replace_dict):
    for col in column_list:
        # Apply the replace function to the specified column using the replace_dict
        df[col] = df[col].replace(replace_dict, regex=True)
    return df

# Define the replacement dictionary
replace_dict = {
    'B.p.l.': 'below poverty line',
    'A.p.l.': 'above poverty line',
    'Bpl': 'below poverty line',
    'Fisheries' : 'Fisheries and Animal Husbandry',
    'Holder' : '(Below poverty line / Above poverty line)'
}


In [9]:
IGNORE_WORDS = ['proportion', 'whose', 'number', 'per', 'within', 'no', 'under', 'volume', 'use', 'type', 'percentage', 'total', 'data', 'year', 'due', 'rate', 'hects', 'hect', 'noof', 'years']

In [10]:
# Creating our tokenizer function
def cleaning(sentence):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    wn = nltk.WordNetLemmatizer()
    ignore_word = []
    cleaned = []
    for i in range(len(sentence)):
        letters_1 = re.sub('[^a-zA-Z.]', ' ', sentence[i])
        letters = re.sub(r'\.', '', letters_1)
        letters = letters.lower().split() 
        letters = [wn.lemmatize(w) for w in letters if w not in stopwords ]
        IGNORE_WORDS_LEM = [wn.lemmatize(w).lower() for w in IGNORE_WORDS]
        letters = [w for w in letters if w not in IGNORE_WORDS_LEM]
        #letters_list = list(set(letters))
        letters_list = list(OrderedDict.fromkeys(letters))   
        cleaned.append(letters_list)

    return cleaned

In [11]:
# Creating our tokenizer function
def concat_cleaning(sentence):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    wn = nltk.WordNetLemmatizer()

    cleaned = []
    for i in range(len(sentence)):
        letters_1 = re.sub('[^a-zA-Z.]', ' ', sentence[i])
        letters = re.sub(r'\.', '', letters_1)
        letters = letters.lower().split() 
        #letters = [wn.lemmatize(w) for w in letters if w not in stopwords ]
        #IGNORE_WORDS_LEM = [wn.lemmatize(w).lower() for w in IGNORE_WORDS]
        #letters = [w for w in letters if w not in IGNORE_WORDS_LEM]
        letters = ' '.join(letters)
        cleaned.append(letters)

    return cleaned

In [12]:
def process_keyword(data):
    result = []
    for sublist in data:
        inner_list = []
        for word, _ in sublist:
            # Split the word if it contains a space and reverse the order of words
            words = word.split()
            # Filter out numeric values
            words = [w for w in words if not w.isdigit()]
            if words:  # Exclude empty lists
                inner_list.append(words)
        inner_list.reverse()
        result.append(inner_list)
    return result

In [13]:
def add_elements(row):
    if not row['keywords']:  # Check if col2 is empty
        row['keywords'].extend([[elem] for elem in row['new_indicators']])  # Add elements from col1 to col2 as lists
    return row

In [14]:
def concatenate_table_name(df):
    for index, row in df.iterrows():
        if ',' not in row['Description']:
            df.at[index, 'Description'] += ', ' + row['Table_name']
        elif row['Description'].count(',') == 1:
            parts = row['Description'].split(',')
            if len(parts) == 2:
                part1, part2 = parts
                if re.match(r'^\w+$', part1.strip()) or re.match(r'^\w+$', part2.strip()):
                    df.at[index, 'Description'] += ', ' + row['Table_name']
    return df


In [15]:
def concatenate_all(df):
    for index, row in df.iterrows():
        df.at[index, 'Description'] += ', ' + row['Table_name'] + ', ' + row['Chapter_name'] 
    return df

In [16]:
missing_words = []
miss_added = []
def sent_vec(row):
    sent = row['new_description']
    cap_tab = row['Chapter_name'] + ", " + row['Table_name']
    vector_size = wv.vector_size
    vectors = []
    miss_w = []
    for w in sent:
        if w in wv:
            wv_res = np.zeros(vector_size)
            wv_res = wv[w]
            vectors.append(wv_res)
        else:
            missing_words.append(("KAG", w))
            miss_w.append(w)
    if len(miss_w)>0:
        vectors.append(missin_word_vec(cap_tab))
        miss_added.append(miss_w)
    vec_arr = np.array(vectors)
    len_vec = len(vectors)
    return vec_arr

In [17]:
def missin_word_vec(sent):
    vector_size = wv.vector_size
    sent_list = cleaning([sent])
    wv_res = np.zeros(vector_size)
    ctr = 0
    for w in sent_list[0]:
        if w in wv:
            wv_res += wv[w]
            ctr += 1
    if ctr>0:
        wv_res = wv_res/ctr
    else:
        wv_res = np.random.rand(vector_size)
        print('yes')
    return wv_res

In [18]:
def sent_vec_avg(sent):
    vector_size = wv.vector_size
    missing_words = []
    vectors = []
    for w in sent:
        ctr = 0
        wv_res = np.zeros(vector_size)
        for word in  w:
            if word in wv:
                wv_res += wv[word]
                ctr += 1
            else:
                missing_words.append(word)
        if ctr>0:
            wv_res = wv_res/ctr
        vectors.append(wv_res)
    vec_arr = np.array(vectors)
    len_vec = len(vectors)
    return vec_arr

In [19]:
def count_common_words(row, df_other):
    set_1 = set(row)
    common_words = []
    for _, other_row in df_other.iterrows():
        set_2 = set(other_row['new_description'])
        common_words.append(len(set_1.intersection(set_2)))
    return common_words

In [20]:
def find_similarity_string(similarity_matrix, threshold, chapter_df, sdg_df):
    top_n_similarities = []
    top_n_attr = []
    top_n_chapter_id = []
    top_n_chapter_name = []
    top_n_table_id = []
    top_n_table_name = []
    top_n_count = []
    top_n_description = []
    
    for index, row in similarity_matrix.iterrows():
        # Filter indices based on threshold value
        top_n_indices = [i for i, value in enumerate(row) if value >= threshold]
        
        # Sort indices based on similarity values
        top_n_indices = sorted(top_n_indices, key=lambda i: row.iloc[i], reverse=True)
        
        # Get top n values, chapters, and chapter names
        values = [row.iloc[i] for i in top_n_indices]
        attr = [similarity_matrix.columns[i] for i in top_n_indices]
        attr_des = [chapter_df['Description'].loc[i] for i in top_n_indices]
        chapter_id = [chapter_df['Chapter_id'].loc[i] for i in top_n_indices]
        chapter_name = [chapter_df['Chapter_name'].loc[i] for i in top_n_indices]   
        table_id = [chapter_df['Table_id'].loc[i] for i in top_n_indices]
        table_name = [chapter_df['Table_name'].loc[i] for i in top_n_indices]   


        # Append to respective lists
        top_n_count.append(len(attr))
        top_n_similarities.append(values)
        top_n_attr.append(attr)
        top_n_description.append(attr_des)
        top_n_chapter_id.append(chapter_id)
        top_n_chapter_name.append(chapter_name)
        top_n_table_id.append(table_id)
        top_n_table_name.append(table_name)
    
    # Assign lists to the SDG DataFrame
    sdg_df['top_n_count'] = top_n_count
    sdg_df['top_n_attr'] = top_n_attr
    sdg_df['top_n_table_id'] = top_n_table_id
    sdg_df['top_n_table_name'] = top_n_table_name
    sdg_df['top_n_chapter_id'] = top_n_chapter_id
    sdg_df['top_n_chapter_name'] = top_n_chapter_name
    sdg_df['top_n_similarities'] = top_n_similarities
    sdg_df['top_n_description'] = top_n_description

    return sdg_df


In [21]:
def process_result_string(result_sdg, sdg_df):
    rows = []
    
    for _, row in result_sdg.iterrows():
        attr_id = row['top_n_attr']
        chapter_id = row['top_n_chapter_id']
        table_id = row['top_n_table_id']
        values = row['top_n_similarities']
        attr_desc = row['top_n_description']
        chapter_name = row['top_n_chapter_name']
        table_name = row['top_n_table_name']
        goal_no = row['Goal No.']
        target_id = row['Target_id']
        indicator_id = row['Indicator_id']
        
        for attr, chap, tab, val, desc, chap_name, tab_name in zip(
            attr_id, chapter_id, table_id, values, attr_desc, chapter_name, table_name
        ):
            rows.append({
                'Attr_id': attr,
                'Chapter_id': chap,
                'Chapter_name': chap_name,
                'Table_id': tab,
                'Table_name': tab_name,
                'Description': desc,
                'top_n_goal_id': goal_no,
                'top_n_target_id': target_id,
                'top_n_indicator_id': indicator_id,
                'top_n_similarities': val
            })

    all_goals = pd.DataFrame(rows)
    
    goal_name = dict(zip(sdg_df['Goal No.'], sdg_df['Goal']))
    target_name = dict(zip(sdg_df['Target_id'], sdg_df['Targets']))
    indicator_name = dict(zip(sdg_df['Indicator_id'], sdg_df['Tentative Indicators']))
    
    all_goals['Goal'] = all_goals['top_n_goal_id'].map(goal_name.get)
    all_goals['Targets'] = all_goals['top_n_target_id'].map(target_name.get)
    all_goals['Tentative Indicators'] = all_goals['top_n_indicator_id'].map(indicator_name.get)
    all_goals = all_goals.loc[:,['Attr_id','Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description', 'Goal', 'Targets', 'Tentative Indicators', 'top_n_goal_id', 'top_n_target_id', 'top_n_indicator_id', 'top_n_similarities']]
    return all_goals


In [22]:
def find_similarity_semantic(sdg_df, att_df, threshold):
    ind_list = sdg_df['vec'].tolist()
    att_list = att_df['vec'].tolist()
    min_similarity_values = []
    top_indices_values = []
    for ind in range(len(ind_list)):
        top_indices_list = []
        top_sim_list = []
        top_min_list = []
        for i in ind_list[ind]:
            max_list = []
            for att in range(len(att_list)):
                similarity_matrix = cosine_similarity([i], att_list[att])
                max_similarity = np.max(similarity_matrix)
                max_list.append(max_similarity)
            # Get the indices of the top 10 max values
            #top_indices = [idx for idx, sim in enumerate(max_list) if sim < 1]
            top_indices = sorted(range(len(max_list)), key=lambda i: max_list[i], reverse=True)[:threshold]
            top_indices_list.append(top_indices)
            top_sim = [max_list[i] for i in top_indices]
            top_sim_list.append(top_sim)
            top_min_list.append(min(top_sim))
            
        # Merge the inner lists of l1 and l2
        merged_index = [item for sublist in top_indices_list for item in sublist]
        merged_sim = [item for sublist in top_sim_list for item in sublist]
        
        # Sort l1 based on the values in l2
        sorted_indices = sorted(range(len(merged_sim)), key=lambda i: merged_sim[i], reverse=True)
        sorted_index = [merged_index[i] for i in sorted_indices]
        final_index_list = list(OrderedDict.fromkeys(sorted_index))
            
        #top_indices_set = list(OrderedDict.fromkeys(top_indices_list))
        top_indices_values.append(final_index_list)
        min_similarity_values.append(top_min_list)
    return min_similarity_values, top_indices_values


In [23]:
#indicator words to attribute complete name
def find_similarity_semantic_threshold(sdg_df, att_df, diff):
    ind_list = sdg_df['vec'].tolist()
    att_list = att_df['vec'].tolist()
    min_list = sdg_df['min_values'].tolist()
    top_indices_values = []
    for ind in range(len(ind_list)):
        top_indices_list = []
        top_sim_list = []
        min_values = min_list[ind]
        for index, i in enumerate(ind_list[ind]):
            max_list = []
            threshold = min_values[index] - diff
            if(threshold <= 0):
                continue
            for att in range(len(att_list)):
                similarity_matrix = cosine_similarity([i], att_list[att])
                max_value = np.max(similarity_matrix)
                max_list.append((max_value, att))
            
            # Get the indices of values strictly greater than threshold
            top_indices = [idx for val, idx in max_list if val > threshold]
            # Sort top_indices based on the corresponding values in max_list
            top_indices.sort(key=lambda x: next((val for val, idx in max_list if idx == x), 0), reverse=True)
            top_indices_list.append(top_indices)
            top_sim = [max_list[i][0] for i in top_indices]
            top_sim_list.append(top_sim)
        # Merge the inner lists of l1 and l2
        merged_index = [item for sublist in top_indices_list for item in sublist]
        merged_sim = [item for sublist in top_sim_list for item in sublist]
        
        # Sort l1 based on the values in l2
        sorted_indices = sorted(range(len(merged_sim)), key=lambda i: merged_sim[i], reverse=True)
        sorted_index = [merged_index[i] for i in sorted_indices]
        final_index_list = list(OrderedDict.fromkeys(sorted_index))
        
        #top_indices_set = list(OrderedDict.fromkeys(top_indices_list))
        top_indices_values.append(final_index_list)
        
    return top_indices_values

In [24]:
def finding_attr_details(sdg_df, att_df):
    result_list = sdg_df['top_n_index'].tolist()
    top_n_count = []
    top_n_attr = []
    top_n_chapter_id = []
    top_n_table_id = []
    top_n_description = []
    top_n_chapter_name = []
    top_n_table_name = []
    for ll in result_list:
        top_n_count.append(len(ll))
        top_n_attr.append([att_df.loc[idx, 'Attr_id'] for idx in ll])
        top_n_chapter_id.append([att_df.loc[idx, 'Chapter_id'] for idx in ll])
        top_n_table_id.append([att_df.loc[idx, 'Table_id'] for idx in ll])
        top_n_description.append([att_df.loc[idx, 'Description'] for idx in ll])
        top_n_chapter_name.append([att_df.loc[idx, 'Chapter_name'] for idx in ll])
        top_n_table_name.append([att_df.loc[idx, 'Table_name'] for idx in ll])
    sdg_df['top_n_count'] = top_n_count
    sdg_df['top_n_attr'] = top_n_attr
    sdg_df['top_n_chapter_id'] = top_n_chapter_id
    sdg_df['top_n_table_id'] = top_n_table_id
    sdg_df['top_n_description'] = top_n_description
    sdg_df['top_n_chapter_name'] = top_n_chapter_name
    sdg_df['top_n_table_name'] = top_n_table_name
    return sdg_df

In [25]:
def process_result_semantic(result_sdg, sdg_df):
    rows = []
    
    for _, row in result_sdg.iterrows():
        attr_id = row['top_n_attr']
        chapter_id = row['top_n_chapter_id']
        table_id = row['top_n_table_id']
        attr_desc = row['top_n_description']
        chapter_name = row['top_n_chapter_name']
        table_name = row['top_n_table_name']
        goal_no = row['Goal No.']
        target_id = row['Target_id']
        indicator_id = row['Indicator_id']
        
        for attr, chap, tab, desc, chap_name, tab_name in zip(
            attr_id, chapter_id, table_id, attr_desc, chapter_name, table_name
        ):
            rows.append({
                'Attr_id': attr,
                'Chapter_id': chap,
                'Chapter_name': chap_name,
                'Table_id': tab,
                'Table_name': tab_name,
                'Description': desc,
                'top_n_goal_id': goal_no,
                'top_n_target_id': target_id,
                'top_n_indicator_id': indicator_id
            })

    all_goals = pd.DataFrame(rows)
    
    goal_name = dict(zip(sdg_df['Goal No.'], sdg_df['Goal']))
    target_name = dict(zip(sdg_df['Target_id'], sdg_df['Targets']))
    indicator_name = dict(zip(sdg_df['Indicator_id'], sdg_df['Tentative Indicators']))
    
    all_goals['Goal'] = all_goals['top_n_goal_id'].map(goal_name.get)
    all_goals['Targets'] = all_goals['top_n_target_id'].map(target_name.get)
    all_goals['Tentative Indicators'] = all_goals['top_n_indicator_id'].map(indicator_name.get)
    all_goals = all_goals.loc[:,['Attr_id','Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description', 'Goal', 'Targets', 'Tentative Indicators', 'top_n_goal_id', 'top_n_target_id', 'top_n_indicator_id']]
    return all_goals


In [26]:
sdg = pd.read_csv(absolute_path_sdg, sep=",", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str})

In [27]:
sdg

,Goal No.,Goal,Nodal Department,Targets,Other Related Major Departments,Tentative Indicators,Target_id,Indicator_id
0,1,End poverty in all its forms everywhere,Rural Development,"By 2030, eradicate extreme poverty for all peo...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population below the interna...,1.1,1.1.1
1,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living below the ...,1.2,1.2.1
2,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...","Proportion of men, women and children of all a...",1.2,1.2.2
3,1,End poverty in all its forms everywhere,Rural Development,Implement nationally appropriate social protec...,"Urban Development, Agriculture, Horticulture, ...",Percentage of the population covered by social...,1.3,1.3.1
4,1,End poverty in all its forms everywhere,Rural Development,"By 2030, ensure that all men and women, in par...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living in househo...,1.4,1.4.1
...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,Finance,"Promote the development, transfer, disseminati...",Environment and Scientific Technology.,Total amount of approved funding to promote th...,16.4,16.4.1
175,16,Strengthen the means of implementation and rev...,Finance,Fully operationalize the technology bank and s...,"Environment and Scientific Technology, Inform...",Proportion of individuals using the Internet.,16.5,16.5.1
176,17,"Data, monitoring and accountability",Finance,"By 2020, enhance capacity-building support to ...","Planning, Finance, Economic and Statistics.",Proportion of sustainable development indicato...,17.1,17.1.1
177,17,"Data, monitoring and accountability",Finance,"By 2030, build on existing initiatives to deve...","Planning, Finance, Economic and Statistics.",Dollar value of all resources made available t...,17.2,17.2.1


In [28]:
sdg.columns

Index(['Goal No.', 'Goal', 'Nodal Department', 'Targets',
       'Other Related Major Departments', 'Tentative Indicators', 'Target_id',
       'Indicator_id'],
      dtype='object')

In [29]:
sdg.drop(['Nodal Department','Other Related Major Departments'],axis=1,inplace=True)

In [30]:
# Remove duplicate rows
sdg = sdg.drop_duplicates()

In [31]:
sdg.reset_index(inplace = True, drop = True)

In [32]:
sdg

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1
...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1
175,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,Proportion of individuals using the Internet.,16.5,16.5.1
176,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",Proportion of sustainable development indicato...,17.1,17.1.1
177,17,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Dollar value of all resources made available t...,17.2,17.2.1


In [33]:
sdg_new = sdg.copy()

In [34]:
texto_data=sdg_new['Tentative Indicators'].tolist()
process_text=cleaning(texto_data)

cleaned_sent = pd.DataFrame({'new_indicators': process_text})
sdg_new['new_indicators']=cleaned_sent

In [35]:
texto_data0=sdg_new['Tentative Indicators'].tolist()
process_text0=concat_cleaning(texto_data0)

cleaned_sent0 = pd.DataFrame({'new_indicators_str': process_text0})
sdg_new['new_indicators_str']=cleaned_sent0

In [36]:
sdg_new

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...
...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...
175,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,Proportion of individuals using the Internet.,16.5,16.5.1,"[individual, using, internet]",proportion of individuals using the internet
176,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",Proportion of sustainable development indicato...,17.1,17.1.1,"[sustainable, development, indicator, produced...",proportion of sustainable development indicato...
177,17,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Dollar value of all resources made available t...,17.2,17.2.1,"[dollar, value, resource, made, available, str...",dollar value of all resources made available t...


In [37]:
data = pd.read_csv(absolute_path_attr, sep=";", dtype={'Attr_id': str, 'Chapter_id': str, 'Table_id':str})

In [38]:
# Replace words in the DataFrame using the dictionary
data = replace_words(data, ['Chapter_name', 'Table_name', 'Description'], replace_dict)

In [39]:
data_new = concatenate_all(data)
data_new

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description
0,3,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Taluks, Taluks Hoblies Grama Panchayath Villag..."
1,4,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Hoblies, Taluks Hoblies Grama Panchayath Villa..."
2,5,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Grama Panchayaths, Taluks Hoblies Grama Pancha..."
3,6,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Circles, Taluks Hoblies Grama Panchayath Villa..."
4,7,1,General Information,1.2,Inhabited Un-inhabited and Total Villages per ...,"Un-Inhabited Inhabited,Villages, Inhabited Un-..."
...,...,...,...,...,...,...
1548,1558,17,Other Information,17.3,Registration of Births and Deaths,"Total,Births,Late Registered events 2019, Regi..."
1549,1559,17,Other Information,17.3,Registration of Births and Deaths,"Male,Deaths,Late Registered events 2019, Regis..."
1550,1560,17,Other Information,17.3,Registration of Births and Deaths,"Female,Deaths,Late Registered events 2019, Reg..."
1551,1561,17,Other Information,17.3,Registration of Births and Deaths,"Transgender,Deaths,Late Registered events 2019..."


In [40]:
# Create a new DataFrame with selected columns
new_df = data_new.copy()
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description
0,3,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Taluks, Taluks Hoblies Grama Panchayath Villag..."
1,4,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Hoblies, Taluks Hoblies Grama Panchayath Villa..."
2,5,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Grama Panchayaths, Taluks Hoblies Grama Pancha..."
3,6,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Circles, Taluks Hoblies Grama Panchayath Villa..."
4,7,1,General Information,1.2,Inhabited Un-inhabited and Total Villages per ...,"Un-Inhabited Inhabited,Villages, Inhabited Un-..."
...,...,...,...,...,...,...
1548,1558,17,Other Information,17.3,Registration of Births and Deaths,"Total,Births,Late Registered events 2019, Regi..."
1549,1559,17,Other Information,17.3,Registration of Births and Deaths,"Male,Deaths,Late Registered events 2019, Regis..."
1550,1560,17,Other Information,17.3,Registration of Births and Deaths,"Female,Deaths,Late Registered events 2019, Reg..."
1551,1561,17,Other Information,17.3,Registration of Births and Deaths,"Transgender,Deaths,Late Registered events 2019..."


In [41]:
texto_data1=new_df['Description'].tolist()
process_text1=cleaning(texto_data1)

cleaned_sent1 = pd.DataFrame({'new_description': process_text1})
new_df['new_description']=cleaned_sent1

In [42]:
texto_data11=new_df['Description'].tolist()
process_text11=concat_cleaning(texto_data11)

cleaned_sent11 = pd.DataFrame({'new_description_str': process_text11})
new_df['new_description_str']=cleaned_sent11

In [43]:
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,new_description,new_description_str
0,3,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Taluks, Taluks Hoblies Grama Panchayath Villag...","[taluks, hoblies, grama, panchayath, village, ...",taluks taluks hoblies grama panchayath village...
1,4,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Hoblies, Taluks Hoblies Grama Panchayath Villa...","[hoblies, taluks, grama, panchayath, village, ...",hoblies taluks hoblies grama panchayath villag...
2,5,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Grama Panchayaths, Taluks Hoblies Grama Pancha...","[grama, panchayaths, taluks, hoblies, panchaya...",grama panchayaths taluks hoblies grama panchay...
3,6,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Circles, Taluks Hoblies Grama Panchayath Villa...","[circle, taluks, hoblies, grama, panchayath, v...",circles taluks hoblies grama panchayath villag...
4,7,1,General Information,1.2,Inhabited Un-inhabited and Total Villages per ...,"Un-Inhabited Inhabited,Villages, Inhabited Un-...","[un, inhabited, village, census, general, info...",un inhabited inhabited villages inhabited un i...
...,...,...,...,...,...,...,...,...
1548,1558,17,Other Information,17.3,Registration of Births and Deaths,"Total,Births,Late Registered events 2019, Regi...","[birth, late, registered, event, registration,...",total births late registered events registrati...
1549,1559,17,Other Information,17.3,Registration of Births and Deaths,"Male,Deaths,Late Registered events 2019, Regis...","[male, death, late, registered, event, registr...",male deaths late registered events registratio...
1550,1560,17,Other Information,17.3,Registration of Births and Deaths,"Female,Deaths,Late Registered events 2019, Reg...","[female, death, late, registered, event, regis...",female deaths late registered events registrat...
1551,1561,17,Other Information,17.3,Registration of Births and Deaths,"Transgender,Deaths,Late Registered events 2019...","[transgender, death, late, registered, event, ...",transgender deaths late registered events regi...


String Matching

In [44]:
common_word_matrix = sdg_new['new_indicators'].apply(count_common_words, args=(new_df,))
common_word_matrix

0      [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, ...
1      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                             ...                        
174    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
175    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
176    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
177    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
178    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: new_indicators, Length: 179, dtype: object

In [45]:
attr_id = new_df['Attr_id'].tolist()
ind_id = sdg_new['Indicator_id'].tolist()

In [46]:
# Determine the maximum number of words in any list
max_words = max(common_word_matrix, key=lambda x: len(x))

# Create DataFrame with columns Word_1, Word_2, ..., Word_n
common_word_df = pd.DataFrame(common_word_matrix.tolist(), columns=attr_id, index=ind_id)

In [47]:
common_word_df

,3,4,5,6,7,8,9,10,11,12,...,1553,1554,1555,1556,1557,1558,1559,1560,1561,1562
1.1.1,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1.2.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.2.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.3.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.4.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16.4.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16.5.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17.1.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17.2.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
# Example usage:
threshold_value = 1  # Define the threshold value

# Call the function to find chapter_ids with similarity greater than the threshold
sdg_string_map = find_similarity_string(common_word_df, threshold_value, new_df, sdg_new.copy())


In [49]:
sdg_string_map

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,top_n_count,top_n_attr,top_n_table_id,top_n_table_name,top_n_chapter_id,top_n_chapter_name,top_n_similarities,top_n_description
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...,598,"[24, 25, 27, 28, 30, 31, 33, 34, 36, 37, 39, 4...","[1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, ...",[Ration Shops and Ration Card (Below poverty l...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General Information, General Information, Gen...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[Urban,Ration shops, Ration Shops and Ration C..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...,521,"[1526, 1529, 1534, 24, 25, 26, 27, 28, 29, 30,...","[17.2, 17.2, 17.2, 1.6, 1.6, 1.6, 1.6, 1.6, 1....",[Number of Pensioners under Social Security Sc...,"[17, 17, 17, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Other Information, Other Information, Other I...","[4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",[Bank Post Office Account (Below poverty line ...
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...,350,"[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[17.2, 10.8, 10.8, 10.8, 10.8, 10.8, 10.8, 10....",[Number of Pensioners under Social Security Sc...,"[17, 10, 10, 10, 10, 10, 10, 10, 13, 13, 13, 1...","[Other Information, Education, Education, Educ...","[3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",[Bank Post Office Account (Below poverty line ...
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...,430,"[117, 118, 119, 120, 121, 122, 123, 1363, 1525...","[2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 13.5, 17.2...","[Child Sex Ratio 0-6 Years, Child Sex Ratio 0-...","[2, 2, 2, 2, 2, 2, 2, 13, 17, 17, 17, 2, 2, 2,...","[Area and Population, Area and Population, Are...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, ...","[Total,Child Sex Ratio 0-6 Years 2001, Child S..."
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...,357,"[70, 71, 72, 73, 74, 75, 76, 77, 278, 279, 280...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.19,...",[Area Population Density and Households 2011 C...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[Area and Population, Area and Population, Are...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[Geograpical Area Sq.Kms, Area Population Dens..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...,79,"[580, 581, 582, 583, 584, 585, 586, 587, 588, ...","[4.18, 4.18, 4.18, 4.18, 4.18, 4.18, 4.18, 4.1...","[Sericulture 2019-20, Sericulture 2019-20, Ser...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 13, 13, 13, ...","[Agriculture, Horticulture & Sericulture, Agri...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..

In [50]:
string_all_goals = process_result_string(sdg_string_map, sdg_new)

In [51]:
sdg_string_map.to_csv(absolute_path_string, sep=";", index=False)

In [52]:
string_all_goals.to_csv(absolute_path_string_unroll, sep=";", index=False)

Semantic Mapping

In [53]:
sdg_sem = sdg_new.copy()

In [54]:
word_count = sdg_sem['new_indicators'].apply(lambda x: len(x))
word_count

0      14
1       9
2      10
3      23
4       6
       ..
174    11
175     3
176    15
177     8
178     3
Name: new_indicators, Length: 179, dtype: int64

In [55]:
from collections import Counter
w_c = Counter(word_count)
w_c.most_common()

[(5, 21),
 (6, 18),
 (7, 17),
 (4, 17),
 (9, 16),
 (10, 15),
 (8, 14),
 (3, 12),
 (12, 8),
 (11, 6),
 (16, 5),
 (2, 5),
 (14, 4),
 (13, 4),
 (15, 4),
 (20, 2),
 (26, 2),
 (17, 2),
 (23, 1),
 (24, 1),
 (22, 1),
 (29, 1),
 (27, 1),
 (21, 1),
 (28, 1)]

In [56]:
w_c.most_common(10)

[(5, 21),
 (6, 18),
 (7, 17),
 (4, 17),
 (9, 16),
 (10, 15),
 (8, 14),
 (3, 12),
 (12, 8),
 (11, 6)]

In [57]:
np.mean(word_count)

8.726256983240223

In [58]:
median_indicator = np.median(word_count)
print(median_indicator)

7.0


In [59]:
from statistics import mode
mode_indicator = mode(word_count)
mode_indicator

5

In [60]:
stopwords_set = list(set(nltk.corpus.stopwords.words('english')))
stopwords_set.extend(IGNORE_WORDS)
stopwords_set = list(set(stopwords_set))
stopwords_set

['doesn',
 'them',
 'has',
 'while',
 'due',
 'he',
 'type',
 'having',
 'weren',
 'my',
 'against',
 'than',
 'nor',
 'now',
 'can',
 "won't",
 'about',
 'why',
 "shouldn't",
 'isn',
 "mightn't",
 'own',
 'yours',
 'up',
 'hadn',
 'ours',
 'whom',
 'not',
 'too',
 'itself',
 'per',
 'of',
 'above',
 'you',
 'further',
 "couldn't",
 'themselves',
 "aren't",
 'years',
 'our',
 'which',
 "you'd",
 'such',
 'shan',
 'yourselves',
 'did',
 "isn't",
 'in',
 'use',
 'hects',
 "should've",
 'below',
 'her',
 'does',
 'didn',
 'yourself',
 'i',
 'volume',
 'these',
 'under',
 'your',
 'that',
 'there',
 've',
 'but',
 'was',
 'during',
 'doing',
 'out',
 'so',
 'when',
 "that'll",
 'o',
 'y',
 "needn't",
 "hadn't",
 'couldn',
 'between',
 'me',
 'ourselves',
 "it's",
 'data',
 'again',
 'each',
 'total',
 'd',
 'mustn',
 "doesn't",
 'll',
 'hers',
 'the',
 'this',
 'had',
 'don',
 "wasn't",
 'how',
 'who',
 "didn't",
 'being',
 'those',
 'noof',
 'a',
 'and',
 'all',
 'or',
 'needn',
 'whose',

In [61]:
#using 'new_indicators_str', takin top_n=6 as median = 8 and nr_candidates=10 as mean = 9.2
sdg_sem['keywords']=kw_model.extract_keywords(sdg_sem['new_indicators_str'].tolist(), keyphrase_ngram_range=(1,2), stop_words = stopwords_set, use_maxsum=True, nr_candidates=10, top_n=6)

In [62]:
sdg_sem['keywords'] = process_keyword(sdg_sem['keywords'].tolist())
sdg_sem

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,keywords
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...,"[[poverty, line], [population], [age, group], ..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...,"[[living, national], [poverty, line], [age, gr..."
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...,"[[according, national], [children, ages], [liv..."
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...,"[[age, persons], [children], [population, cove..."
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...,"[[basic, services], [population], [living], [a..."
...,...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...,"[[development], [sound, technologies], [promot..."
175,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,Proportion of individuals using the Internet.,16.5,16.5.1,"[individual, using, internet]",proportion of individuals using the internet,[]
176,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",Proportion of sustainable development indicato...,17.1,17.1.1,"[sustainable, development, indicator, produced...",proportion of sustainable development indicato...,"[[official, statistics], [level], [target, acc..."
177,17,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Dollar value of all resources made available t...,17.2,17.2.1,"[dollar, value, resource, made, available, str...",dollar value of all resources made available t...,"[[dollar, value], [statistical, capacity], [va..."


In [63]:
# Apply the function to each row of the DataFrame
sdg_sem = sdg_sem.apply(add_elements, axis=1)
sdg_sem

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,keywords
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...,"[[poverty, line], [population], [age, group], ..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...,"[[living, national], [poverty, line], [age, gr..."
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...,"[[according, national], [children, ages], [liv..."
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...,"[[age, persons], [children], [population, cove..."
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...,"[[basic, services], [population], [living], [a..."
...,...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...,"[[development], [sound, technologies], [promot..."
175,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,Proportion of individuals using the Internet.,16.5,16.5.1,"[individual, using, internet]",proportion of individuals using the internet,"[[individual], [using], [internet]]"
176,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",Proportion of sustainable development indicato...,17.1,17.1.1,"[sustainable, development, indicator, produced...",proportion of sustainable development indicato...,"[[official, statistics], [level], [target, acc..."
177,17,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Dollar value of all resources made available t...,17.2,17.2.1,"[dollar, value, resource, made, available, str...",dollar value of all resources made available t...,"[[dollar, value], [statistical, capacity], [va..."


In [64]:
sdg_sem['vec'] = sdg_sem['keywords'].apply(sent_vec_avg)
sdg_sem

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,keywords,vec
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...,"[[poverty, line], [population], [age, group], ...","[[-0.07521600276231766, 0.474590003490448, 0.2..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...,"[[living, national], [poverty, line], [age, gr...","[[-0.21916300430893898, 0.3655640622600913, 0...."
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...,"[[according, national], [children, ages], [liv...","[[-0.17308800294995308, 0.43849001079797745, 0..."
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...,"[[age, persons], [children], [population, cove...","[[-0.5623050034046173, 0.039556750562042, 0.20..."
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...,"[[basic, services], [population], [living], [a...","[[-0.6532500088214874, -0.4114450067281723, -0..."
...,...,...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...,"[[development], [sound, technologies], [promot...","[[-0.10136999934911728, -0.27173998951911926, ..."
175,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,Proportion of individuals using the Internet.,16.5,16.5.1,"[individual, using, internet]",proportion of individuals using the internet,"[[individual], [using], [internet]]","[[-0.5918700098991394, 0.49957001209259033, -0..."
176,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",Proportion of sustainable development indicato...,17.1,17.1.1,"[sustainable, development, indicator, produced...",proportion of sustainable development indicato...,"[[official, statistics], [level], [target, acc...","[[-0.10437500174157321, 0.33179498463869095, 0..."
177,17,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Dollar value of all resources made available t...,17.2,17.2.1,"[dollar, value, resource, made, available, str...",dollar value of all resources made available t...,"[[dollar, value], [statistical, capacity], [va...","[[-0.328249990940094, 0.2354124989360571, -0.1..."


In [65]:
new_df['vec'] = new_df.apply(sent_vec, axis=1)

In [66]:
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,new_description,new_description_str,vec
0,3,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Taluks, Taluks Hoblies Grama Panchayath Villag...","[taluks, hoblies, grama, panchayath, village, ...",taluks taluks hoblies grama panchayath village...,"[[-0.18463000655174255, -1.0946999788284302, -..."
1,4,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Hoblies, Taluks Hoblies Grama Panchayath Villa...","[hoblies, taluks, grama, panchayath, village, ...",hoblies taluks hoblies grama panchayath villag...,"[[-0.18463000655174255, -1.0946999788284302, -..."
2,5,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Grama Panchayaths, Taluks Hoblies Grama Pancha...","[grama, panchayaths, taluks, hoblies, panchaya...",grama panchayaths taluks hoblies grama panchay...,"[[0.09846500307321548, -0.4200499951839447, -0..."
3,6,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,"Circles, Taluks Hoblies Grama Panchayath Villa...","[circle, taluks, hoblies, grama, panchayath, v...",circles taluks hoblies grama panchayath villag...,"[[0.15791000425815582, 0.24285000562667847, -0..."
4,7,1,General Information,1.2,Inhabited Un-inhabited and Total Villages per ...,"Un-Inhabited Inhabited,Villages, Inhabited Un-...","[un, inhabited, village, census, general, info...",un inhabited inhabited villages inhabited un i...,"[[0.44261, -0.17972, -0.27015, -0.39474, -0.03..."
...,...,...,...,...,...,...,...,...,...
1548,1558,17,Other Information,17.3,Registration of Births and Deaths,"Total,Births,Late Registered events 2019, Regi...","[birth, late, registered, event, registration,...",total births late registered events registrati...,"[[-0.11292, -0.33504, -0.24223, 0.2406, -0.285..."
1549,1559,17,Other Information,17.3,Registration of Births and Deaths,"Male,Deaths,Late Registered events 2019, Regis...","[male, death, late, registered, event, registr...",male deaths late registered events registratio...,"[[-0.32418, 0.4799, -0.17283, -0.2947, -0.0892..."
1550,1560,17,Other Information,17.3,Registration of Births and Deaths,"Female,Deaths,Late Registered events 2019, Reg...","[female, death, late, registered, event, regis...",female deaths late registered events registrat...,"[[-0.2299, 0.42912, -0.25127, -0.19251, -0.262..."
1551,1561,17,Other Information,17.3,Registration of Births and Deaths,"Transgender,Deaths,Late Registered events 2019...","[transgender, death, late, registered, event, ...",transgender deaths late registered events regi...,"[[-0.058667, -0.21876, 0.050555, 0.34848, -0.1..."


In [67]:
missing_words_set = list(set(missing_words))
print(len(missing_words_set), missing_words_set)

85 [('KAG', 'bhagyalakshmi'), ('KAG', 'mulbery'), ('KAG', 'householdsdemanded'), ('KAG', 'arecanuts'), ('KAG', 'marketin'), ('KAG', 'sapota'), ('KAG', 'assoistance'), ('KAG', 'oftotal'), ('KAG', 'geograpical'), ('KAG', 'ccusers'), ('KAG', 'anthyodaya'), ('KAG', 'handpump'), ('KAG', 'ksdc'), ('KAG', 'tubectomy'), ('KAG', 'sqkms'), ('KAG', 'priva'), ('KAG', 'lpcd'), ('KAG', 'suraksha'), ('KAG', 'vehiclea'), ('KAG', 'kutirajyoti'), ('KAG', 'bufaloes'), ('KAG', 'nddp'), ('KAG', 'carriag'), ('KAG', 'pwss'), ('KAG', 'oranage'), ('KAG', 'pmay'), ('KAG', 'brambedkar'), ('KAG', 'raingauge'), ('KAG', 'pradhana'), ('KAG', 'raddish'), ('KAG', 'autonom'), ('KAG', 'areaunder'), ('KAG', 'anganawadis'), ('KAG', 'tubewell'), ('KAG', 'swachh'), ('KAG', 'socie'), ('KAG', 'sericulturist'), ('KAG', 'lepros'), ('KAG', 'vacanthouses'), ('KAG', 'nrlm'), ('KAG', 'multiaxled'), ('KAG', 'rsin'), ('KAG', 'anganawadi'), ('KAG', 'mahathma'), ('KAG', 'sesamum'), ('KAG', 'employement'), ('KAG', 'madilu'), ('KAG', 'he

In [68]:
miss_added_set = list(set([item for sublist in miss_added for item in sublist]))
print(len(miss_added_set), miss_added_set)

85 ['bottel', 'handpump', 'oranage', 'gddp', 'anganawadi', 'manaswini', 'sapota', 'airculated', 'geograpical', 'brambedkar', 'matsyashraya', 'sinchayee', 'kaccha', 'beneficiries', 'panchayth', 'hepatities', 'socie', 'allopathy', 'lepros', 'vajapayee', 'delivaries', 'nivasa', 'nddp', 'mythri', 'ksdc', 'pomogranate', 'bankwise', 'swachh', 'mgnrega', 'uncultivatable', 'vehiclea', 'yojane', 'householdsdemanded', 'anthyodaya', 'borewells', 'hectar', 'autonom', 'bhagyajyoti', 'fishlets', 'estp', 'vikasa', 'tubewell', 'multiaxled', 'carriag', 'anganawadis', 'sesamum', 'bufaloes', 'suraksha', 'tubectomy', 'lpcd', 'vacanthouses', 'sectorwise', 'pradhana', 'phisical', 'mtonnes', 'avare', 'kutirajyoti', 'cmkky', 'poulation', 'sterlisation', 'arecanuts', 'ccusers', 'kscard', 'madilu', 'assoistance', 'areaunder', 'hoblies', 'teachersal', 'marketin', 'sericulturist', 'pwss', 'mulbery', 'raingauge', 'priva', 'mahathma', 'sqkms', 'raddish', 'bhagyalakshmi', 'rsin', 'socities', 'pmay', 'oftotal', 'nrlm

In [69]:
sdg_count_threshold = sdg_sem.copy()

In [70]:
min_val_list, top_n_index_list = find_similarity_semantic(sdg_count_threshold, new_df, 10)

In [71]:
sdg_count_threshold['top_n_index'] = top_n_index_list
sdg_count_threshold['min_values'] = min_val_list
sdg_count_threshold

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,keywords,vec,top_n_index,min_values
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...,"[[poverty, line], [population], [age, group], ...","[[-0.07521600276231766, 0.474590003490448, 0.2...","[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 22, 2...","[0.8120515928906179, 1.0000000000000009, 0.794..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...,"[[living, national], [poverty, line], [age, gr...","[[-0.21916300430893898, 0.3655640622600913, 0....","[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 21, 2...","[0.7903468200598074, 0.8120515928906179, 0.794..."
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...,"[[according, national], [children, ages], [liv...","[[-0.17308800294995308, 0.43849001079797745, 0...","[996, 1366, 1367, 1379, 1380, 1381, 1382, 1383...","[0.8887401699967143, 0.6779166506008225, 0.885..."
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...,"[[age, persons], [children], [population, cove...","[[-0.5623050034046173, 0.039556750562042, 0.20...","[1214, 1215, 1216, 1217, 1218, 1219, 1220, 122...","[0.8303263734472492, 0.7450329169830372, 0.835..."
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...,"[[basic, services], [population], [living], [a...","[[-0.6532500088214874, -0.4114450067281723, -0...","[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 1001,...","[0.6732416697190915, 1.0000000000000009, 0.524..."
...,...,...,...,...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...,"[[development], [sound, technologies], [promot...","[[-0.10136999934911728, -0.27173998951911926, ...","[570, 571, 572, 573, 574, 575, 576, 577, 1331,...","[1.0000000000000009, 0.47943460610968414, 0.56..."
175,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,Proportion of individuals using the Internet.,16.5,16.5.1,"[individual, using, internet]",proportion of individuals using the internet,"[[individual], [using], [internet]]","[[-0.5918700098991394, 0.49957001209259033, -0...","[1373, 1374, 1012, 1421, 1422, 1423, 1424, 142...","[0.5072592691296408, 0.5684845828500003, 0.567..."
176,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",Proportion of sustainable development indicato...,17.1,17.1.1,"[sustainable, development, indicator, produced...",proportion of sustainable development indicato...,"[[official, statistics], [level], [target, acc...","[[-0.10437500174157321, 0.33179498463869095, 0...","[570, 571, 572, 573, 574, 575, 576, 577, 1331,...","[0.5132201530035716, 0.5584521024536246, 0.456..."
177,17,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...

In [72]:
sdg_min_value = sdg_count_threshold.copy()

In [73]:
sdg_count_threshold_all = finding_attr_details(sdg_count_threshold, new_df)
sdg_count_threshold_all

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,keywords,vec,top_n_index,min_values,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...,"[[poverty, line], [population], [age, group], ...","[[-0.07521600276231766, 0.474590003490448, 0.2...","[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 22, 2...","[0.8120515928906179, 1.0000000000000009, 0.794...",57,"[70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 25, 2...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, ...","[Geograpical Area Sq.Kms, Area Population Dens...","[Area and Population, Area and Population, Are...",[Area Population Density and Households 2011 C...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...,"[[living, national], [poverty, line], [age, gr...","[[-0.21916300430893898, 0.3655640622600913, 0....","[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 21, 2...","[0.7903468200598074, 0.8120515928906179, 0.794...",50,"[70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 24, 2...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, ...","[Geograpical Area Sq.Kms, Area Population Dens...","[Area and Population, Area and Population, Are...",[Area Population Density and Households 2011 C...
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...,"[[according, national], [children, ages], [liv...","[[-0.17308800294995308, 0.43849001079797745, 0...","[996, 1366, 1367, 1379, 1380, 1381, 1382, 1383...","[0.8887401699967143, 0.6779166506008225, 0.885...",59,"[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[9, 14, 14, 14, 14, 14, 14, 14, 14, 14, 1, 1, ...","[9.2, 14.1, 14.1, 14.3, 14.3, 14.3, 14.3, 14.3...","[National Highway, P.w.d Road Length on 31-03-...","[Transport and Communication, Rural Developmen...",[P.w.d Road Length on 31-03-2020 In Kms. Cumul...
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...,"[[age, persons], [children], [population, cove...","[[-0.5623050034046173, 0.039556750562042, 0.20...","[1214, 1215, 1216, 1217, 1218, 1219, 1220, 122...","[0.8303263734472492, 0.7450329169830372, 0.835...",58,"[1224, 1225, 1226, 1227, 1228, 1229, 1230, 123...","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 2, 2,...","[12.1, 12.1, 12.1, 12.1, 12.1, 12.1, 12.1, 12....","[Nos,Boys Hostels,Government Pre-Matric, Sched...","[Social Welfare, Social Welfare, Social Welfar...",[Scheduled Castes Students Hostels on 31-3-202...
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...,"[[basic, services], [population], [living], [a...","[[-0.6532500088214874, -0.4114450067281723, -0...","[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 1001,...","[0.6732416697190915, 1.0000000000000009, 0.524...",56,"[70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 1011,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 9, 9, 9, 9, 9, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, ...","[Geograpical

In [74]:
sdg_copy = sdg_sem.copy()

In [75]:
sdg_count_threshold_all_goals = process_result_semantic(sdg_count_threshold_all, sdg_copy)

In [76]:
sdg_count_threshold_all.to_csv(absolute_path_semantic, sep=";", index=False)

In [77]:
sdg_count_threshold_all_goals.to_csv(absolute_path_semantic_unroll, sep=";", index=False)

In [78]:
sdg_count_threshold_all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,Tentative Indicators,top_n_goal_id,top_n_target_id,top_n_indicator_id
0,70,2,Area and Population,2.1,Area Population Density and Households 2011 Ce...,"Geograpical Area Sq.Kms, Area Population Densi...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
1,71,2,Area and Population,2.1,Area Population Density and Households 2011 Ce...,"Population Density, Area Population Density an...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
2,72,2,Area and Population,2.1,Area Population Density and Households 2011 Ce...,"Rual,Households, Area Population Density and H...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
3,73,2,Area and Population,2.1,Area Population Density and Households 2011 Ce...,"Urban,Households, Area Population Density and ...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
4,74,2,Area and Population,2.1,Area Population Density and Households 2011 Ce...,"Total,Households, Area Population Density and ...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
...,...,...,...,...,...,...,...,...,...,...,...,...
8310,1393,14,Rural Development and Panchayat Raj,14.3,Phisical Progress under Mahathma Gandhi Nation...,"Female,Man days Generated In Lakhs, Phisical P...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
8311,1394,14,Rural Development and Panchayat Raj,14.3,Phisical Progress under Mahathma Gandhi Nation...,"Total,Man days Generated In Lakhs, Phisical Pr...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
8312,1376,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,"Ddu-gky,Nrlm National Rural Livelihood Mission...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
8313,1377,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,"Cif Community Investment Fund to groups,Nrlm N...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2


In [79]:
sdg_count_sim = sdg_sem.copy()

In [80]:
top_index = find_similarity_semantic_threshold(sdg_min_value, new_df, 0.025)

In [81]:
sdg_count_sim['top_n_index'] = top_index

In [82]:
sdg_count_sim_all = finding_attr_details(sdg_count_sim, new_df)
sdg_count_sim_all

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,keywords,vec,top_n_index,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...,"[[poverty, line], [population], [age, group], ...","[[-0.07521600276231766, 0.474590003490448, 0.2...","[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 7...",608,"[70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 8...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, ...","[Geograpical Area Sq.Kms, Area Population Dens...","[Area and Population, Area and Population, Are...",[Area Population Density and Households 2011 C...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...,"[[living, national], [poverty, line], [age, gr...","[[-0.21916300430893898, 0.3655640622600913, 0....","[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 7...",520,"[70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 8...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, ...","[Geograpical Area Sq.Kms, Area Population Dens...","[Area and Population, Area and Population, Are...",[Area Population Density and Households 2011 C...
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...,"[[according, national], [children, ages], [liv...","[[-0.17308800294995308, 0.43849001079797745, 0...","[996, 1366, 1367, 1379, 1380, 1381, 1382, 1383...",641,"[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[9, 14, 14, 14, 14, 14, 14, 14, 14, 14, 17, 17...","[9.2, 14.1, 14.1, 14.3, 14.3, 14.3, 14.3, 14.3...","[National Highway, P.w.d Road Length on 31-03-...","[Transport and Communication, Rural Developmen...",[P.w.d Road Length on 31-03-2020 In Kms. Cumul...
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...,"[[age, persons], [children], [population, cove...","[[-0.5623050034046173, 0.039556750562042, 0.20...","[1214, 1215, 1216, 1217, 1218, 1219, 1220, 122...",705,"[1224, 1225, 1226, 1227, 1228, 1229, 1230, 123...","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 1...","[12.1, 12.1, 12.1, 12.1, 12.1, 12.1, 12.1, 12....","[Nos,Boys Hostels,Government Pre-Matric, Sched...","[Social Welfare, Social Welfare, Social Welfar...",[Scheduled Castes Students Hostels on 31-3-202...
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...,"[[basic, services], [population], [living], [a...","[[-0.6532500088214874, -0.4114450067281723, -0...","[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 7...",716,"[70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 8...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, ...","[Geograpical Area Sq.Kms, Area Population Dens...","[Area and Population, Area and Population, Are...",[Area Population Density and Households 2011 C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and

In [83]:
sdg_copy2 = sdg_sem.copy()

In [84]:
sdg_count_sim_all_goals = process_result_semantic(sdg_count_sim_all, sdg_copy2)

In [85]:
sdg_count_sim_all.to_csv(absolute_path_sem_threshold, sep=";", index=False)

In [86]:
sdg_count_sim_all_goals.to_csv(absolute_path_sem_threshold_unroll, sep=";", index=False)

In [87]:
sdg_count_sim_all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,Tentative Indicators,top_n_goal_id,top_n_target_id,top_n_indicator_id
0,70,2,Area and Population,2.1,Area Population Density and Households 2011 Ce...,"Geograpical Area Sq.Kms, Area Population Densi...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
1,71,2,Area and Population,2.1,Area Population Density and Households 2011 Ce...,"Population Density, Area Population Density an...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
2,72,2,Area and Population,2.1,Area Population Density and Households 2011 Ce...,"Rual,Households, Area Population Density and H...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
3,73,2,Area and Population,2.1,Area Population Density and Households 2011 Ce...,"Urban,Households, Area Population Density and ...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
4,74,2,Area and Population,2.1,Area Population Density and Households 2011 Ce...,"Total,Households, Area Population Density and ...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
...,...,...,...,...,...,...,...,...,...,...,...,...
59581,1382,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,"Women Groups,Ksdc Karnataka Skill Development ...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
59582,1379,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,"Persons Skilled,Self Employment Programme,Empl...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
59583,1380,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,"Groups Skilled,Self Employment Programme,Emplo...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
59584,1381,14,Rural Development and Panchayat Raj,14.1,Skill Development and Self Employment at the e...,Chief Minister Kaushalya Karnataka Yojane Cmkk...,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
